In [140]:
import pandas as pd

declarations = pd.read_csv('compliance_declaration.csv')
divisions= pd.read_csv('compliance_division.csv')
taxregs = pd.read_csv('compliance_taxregistration.csv')
users = pd.read_csv('auth_user.csv')
countries = pd.read_csv('compliance_country.csv')



In [227]:
# Convert country id to country name
def get_ctry_name(id):
    return countries[countries.id==id].name

def get_div_code(id):
    return divisions[divisions.id==id].code

def get_user_first(id):
    return users[users.id==id].first_name

# get country_id from tax reg id
def get_ctry_id(tax_id):
    country = taxregs[taxregs.id==tax_id].country_id.iloc[0]
    return country
    
    
get_ctry_id(11)



2

In [81]:
taxregs[taxregs.id==11]

,id,tax_id,country_id,division_id
0,11,IE12858131J,2,8


In [48]:
import datetime

new_declaration = {
    'id': 2,
    'declaration_type': 'VAT',
    'month': 'February',
    'period_start': '',
    'period_end': '',
    'payable_receivable_amount': 123124,
    'time_budget': 30,
    'country_id': 2,
    'preparer_id':3, 
    'tax_registration_id':3,
    'year': 2021
    
}



,id,declaration_type,month,period_start,period_end,is_payable,payable_receivable_amount,time_budget,is_submitted_for_review,is_reviewed,is_filed,is_paid,status,completion_pct,country_id,preparer_id,reviewer_id,tax_registration_id,year
0,1,VAT,January,2021-02-01,2021-02-28,0.0,14333398,240,0.0,0.0,0.0,0.0,Started,10.0,2,2,1.0,3,2021
1,2,VAT,February,,,NaN,123124,30,NaN,NaN,NaN,NaN,NaN,NaN,2,3,NaN,3,2021


In [49]:
taxregs

,id,tax_id,country_id,division_id
0,11,IE12858131J,2,8
1,12,FR14297623K,3,8
2,13,ES74982314G,10,8
3,14,DE24532525K,6,8
4,15,IE24424999K,2,7
5,16,IE98713987G,2,6
6,17,IE99821324H,2,5
7,18,IE56569876D,2,4
8,19,FR552298765G,3,9
9,20,FR981233665H,3,10


In [254]:
def generate_decl_details(tax_reg_id,preparer_id, mean_payable, month, year):
  
    declaration_id = len(declarations) +1
    month = month
    year = year

    tax_registration_id = tax_reg_id
    country_id = get_ctry_id(tax_registration_id)
    preparer = preparer_id
    reviewer_id = '5'

    mean_payable= mean_payable
    std_payable= mean_payable/5
    payable_receivable_amount= round(gauss(mean_payable, std_payable),2)

    if payable_receivable_amount > 0:
        is_payable = True
    else:
        is_payable= False

    start = datetime.date(year,month,1)
    end = datetime.date(year, month, calendar.monthrange(year, month)[1])
    time_budget= 240


    new_declaration = {
        'id': declaration_id,
        'declaration_type': 'VAT',
        'month': month,
        'year': year,
        'period_start': start,
        'period_end': end,
        'payable_receivable_amount': payable_receivable_amount,
        'time_budget': time_budget ,
        'country_id': country_id,
        'preparer_id':preparer, 
        'tax_registration_id':tax_registration_id,
        'is_submitted_for_review': True, 
        'is_reviewed': True,
        'is_filed': True, 
        'is_paid': True,
        'is_payable': is_payable,
        'status': 'Filed and Paid',
        'completion_pct': 100,
        'reviewer_id': reviewer_id

    }
    
    return new_declaration



In [255]:
def generate_for_10_years(declarations, tax_reg_id, preparer_id, mean_payable):
    declaration_dict={}
    for i in range (2010, 2020):
        for j in range(12):
            month = j+1
            year = i+1
            round(gauss(mean_payable, mean_payable/3),2)
            declaration_dict[f"{month}-{year}-{tax_reg_id}"] = generate_decl_details(tax_reg_id, preparer_id, mean_payable, month, year) 
    return declaration_dict


In [260]:
declarations = pd.read_csv('compliance_declaration.csv')
df = declarations

for i in range(len(taxregs)):
    preparer = taxregs.iloc[i].preparer
    tax_id = taxregs.iloc[i].id
    mean_payable= round(gauss(50000, 4000000),2)
    new_dict=generate_for_10_years(declarations, tax_id, preparer, mean_payable)
    new_df = pd.DataFrame.from_dict(new_dict, orient='index')
    df = pd.concat([df, new_df], ignore_index=True)

pd.options.display.float_format = '{:,}'.format
df.describe()

,id,is_payable,payable_receivable_amount,time_budget,is_submitted_for_review,is_reviewed,is_filed,is_paid,completion_pct,country_id,preparer_id,tax_registration_id,year
count,"2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0","2,281.0"
mean,1.9995615957913195,0.526085050416484,"371,666.2772687442",240.0,0.9995615957913196,0.9995615957913196,0.9995615957913196,0.9995615957913196,99.96054362121876,4.735642262165717,4.104340201665936,19.992547128452433,"2,015.5024112231476"
std,0.020938104228425708,0.49942859432899067,"4,142,377.3555843597",0.0,0.02093810422842571,0.02093810422842571,0.02093810422842571,0.02093810422842571,1.8844293805581707,2.692549196807895,1.889394109696514,5.488779355768336,2.87458896667204
min,1.0,0.0,"-7,884,213.94",240.0,0.0,0.0,0.0,0.0,10.0,1.0,2.0,3.0,"2,011.0"
25%,2.0,0.0,"-3,450,575.47",240.0,1.0,1.0,1.0,1.0,100.0,2.0,2.0,15.0,"2,013.0"
50%,2.0,1.0,"1,113,121.12",240.0,1.0,1.0,1.0,1.0,100.0,5.0,3.0,20.0,"2,016.0"
75%,2.0,1.0,"3,523,089.15",240.0,1.0,1.0,1.0,1.0,100.0,7.0,6.0,25.0,"2,018.0"
max,2.0,1.0,"14,333,398.0",240.0,1.0,1.0,1.0,1.0,100.0,10.0,7.0,29.0,"2,021.0"


In [261]:
df.sort_values(by='payable_receivable_amount')

,id,declaration_type,month,period_start,period_end,is_payable,payable_receivable_amount,time_budget,is_submitted_for_review,is_reviewed,is_filed,is_paid,status,completion_pct,country_id,preparer_id,reviewer_id,tax_registration_id,year
689,2,VAT,5,2018-05-01,2018-05-31,0,"-7,884,213.94",240,1,1,1,1,Filed and Paid,100,2,3,5,16,2018
716,2,VAT,8,2020-08-01,2020-08-31,0,"-7,696,699.65",240,1,1,1,1,Filed and Paid,100,2,3,5,16,2020
2176,2,VAT,4,2012-04-01,2012-04-30,0,"-7,455,374.44",240,1,1,1,1,Filed and Paid,100,1,7,5,29,2012
659,2,VAT,11,2015-11-01,2015-11-30,0,"-7,420,710.55",240,1,1,1,1,Filed and Paid,100,2,3,5,16,2015
691,2,VAT,7,2018-07-01,2018-07-31,0,"-7,305,081.14",240,1,1,1,1,Filed and Paid,100,2,3,5,16,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2,VAT,10,2020-10-01,2020-10-31,1,"12,193,398.03",240,1,1,1,1,Filed and Paid,100,2,3,5,15,2020
584,2,VAT,8,2019-08-01,2019-08-31,1,"12,213,891.86",240,1,1,1,1,Filed and Paid,100,2,3,5,15,2019
510,2,VAT,6,2013-06-01,2013-06-30,1,"12,371,996.7",240,1,1,1,1,Filed and Paid,100,2,3,5,15,2013
490,2,VAT,10,2011-10-01,2011-10-31,1,"12,806,258.5",240,1,1,1,1,Filed and Paid,100,2,3,5,15,2011


In [251]:
df.head()

,id,declaration_type,month,period_start,period_end,is_payable,payable_receivable_amount,time_budget,is_submitted_for_review,is_reviewed,is_filed,is_paid,status,completion_pct,country_id,preparer_id,reviewer_id,tax_registration_id,year
0,1,VAT,January,2021-02-01,2021-02-28,0,"14,333,398.0",240,0,0,0,0,Started,10,2,2,1,3,2021
1,2,VAT,1,2011-01-01,2011-01-31,1,"60,673.9",240,1,1,1,1,Filed and Paid,100,2,2,5,11,2011
2,2,VAT,2,2011-02-01,2011-02-28,1,"69,608.91",240,1,1,1,1,Filed and Paid,100,2,2,5,11,2011
3,2,VAT,3,2011-03-01,2011-03-31,1,"57,705.99",240,1,1,1,1,Filed and Paid,100,2,2,5,11,2011
4,2,VAT,4,2011-04-01,2011-04-30,1,"59,992.42",240,1,1,1,1,Filed and Paid,100,2,2,5,11,2011
